In [5]:
from time import sleep
import numpy as np
import pandas as pd
import GetOldTweets3 as got

In [6]:
#limit of 16k requests per 15 minutes. Sleep necessary to automate pipeline

def rest(pause_len):
    for i in range(pause_len):
        print(str(pause_len-i),end= ' minutes to next batch')
        for _ in range(4):
            print(end='.')
            sleep(15)
        print()

def scrape(dates):
    print('starting')
    pause_len = 16
    day_max = 9000
    for ctr, date in enumerate(dates[:-1]):
        try:
            # Search for #covid19 tweets on a given date
            tweetCriteria = got.manager.TweetCriteria().setQuerySearch('covid-19')\
                                        .setMaxTweets(day_max)\
                                        .setSince(date)\
                                        .setUntil(dates[ctr+1])\
                                        .setTopTweets(True)
            print('obtained tweets')
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)

            # Convert Tweets to DF
            if len(tweets) > 0:    
                df =  pd.DataFrame([[tweet.id, tweet.permalink, tweet.text, tweet.username, 
                        tweet.date, tweet.to, tweet.mentions, tweet.hashtags, 
                        tweet.retweets, tweet.favorites, tweet.geo] for tweet in tweets])
                df.columns = ['id','url','raw_text','username','datetime_tweet','to','mentions'
                              ,'hashtags','retweets','favorites','geo']
                # rm commas so it doesnt scre up csv later. 
                df['raw_text'] = df['raw_text'].apply(lambda x: x.replace(',',''))
                # Create text column without search hashtag to prevent data leak 
                df['clean_text'] = df['raw_text'].apply(lambda x: x.lower())
                # save df to csv. Rest if appicable, then query another day.
                filename = date+'.csv'
                df.to_csv(filename)
                print(date, 'complete.',len(df),'tweets.')
            else:
                # Keep list of days that had no results. Just in case.
                print(date, 'complete. No Tweets found in search. !# covid-19')
            # write days w no results to txt file. 
        # too many requests or GoT3 down? Wait and try again.
        except:
            print('ERROR, retrying', date)
            print(e)
            rest(pause_len)
            return scrape(dates[ctr:])
    return '***** ALL DATA ACQUIRED! *****'

In [7]:
# # generate list of dates
# days_raw = [31,29,31,30,31,30,31,31,1] # total days of months between 1/1/2020 - 9/1/2020
# dates = []
# for month_num, month_days in enumerate(days_raw):
#     month = []
#     for i in range(month_days):
#         month.append('2020-{:02d}'.format(month_num+9-len(days_raw))+'-{:02d}'.format(i+1))
#     dates = dates + month

# scrape(dates)

In [8]:
scrape(['2020-06-23','2020-06-24'])
scrape(['2020-04-29','2020-04-30'])

starting
obtained tweets
An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=covid-19%20since%3A2020-06-23%20until%3A2020-06-24&src=typd
ERROR, retrying 2020-06-23

16 minutes to next batch....
15 minutes to next batch....
14 minutes to next batch....
13 minutes to next batch....
12 minutes to next batch....
11 minutes to next batch....
10 minutes to next batch....
9 minutes to next batch....
8 minutes to next batch....
7 minutes to next batch....
6 minutes to next batch....
5 minutes to next batch....
4 minutes to next batch....
3 minutes to next batch....
2 minutes to next batch....
1 minutes to next batch....
starting
obtained tweets
2020-06-23 complete. 9000 tweets.
starting
obtained tweets
An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=covid-19%20since%3A2020-04-29%20until%3A2020-04-30&src=typd
ERROR, retrying 2020-04-29

16 m

'***** ALL DATA ACQUIRED! *****'